In [5]:
#import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
#read data
train_data = pd.read_csv("train.csv")

#### 데이터 전처리

데이터 전처리 후에 editedTrain.csv 파일로 저장됨

In [7]:
# 국가를 대륙으로 매핑하는 딕셔너리
country_continent_dict = {
 'US': 'US',
 'Cuba': 'North America',
 'Portugal': 'Europe',
 'Mexico': 'North America',
 'Unknown': 'US',
 'Puerto-Rico': 'North America',
 'Germany': 'Europe',
 'Japan': 'Asia',
 'Poland': 'Europe',
 'Columbia': 'South America',
 'Philippines': 'Asia',
 'Italy': 'Europe',
 'Trinadad&Tobago': 'South America',
 'England': 'Europe',
 'South Korea': 'Asia',
 'Iran': 'Asia',
 'France': 'Europe',
 'India': 'Asia',
 'China': 'Asia',
 'Dominican-Republic': 'North America',
 'Scotland': 'Europe',
 'Ecuador': 'South America',
 'Nicaragua': 'North America',
 'Peru': 'South America',
 'Cambodia': 'Asia',
 'Canada': 'North America',
 'Jamaica': 'North America',
 'Vietnam': 'Asia',
 'Hong Kong': 'Asia',
 'Thailand': 'Asia',
 'Haiti': 'North America',
 'Guatemala': 'North America',
 'Laos': 'Asia',
 'Yugoslavia': 'Europe',
 'Ireland': 'Europe',
 'El-Salvador': 'North America',
 'Panama': 'North America',
 'Honduras': 'North America',
 'Greece': 'Europe',
 'Outlying-U S (Guam USVI etc)': 'US',
 'Hungary': 'Europe',
 'Taiwan': 'Asia',
 'Holand-Netherlands': 'Europe'
}

# 업데이트할 열 목록: 본인 출신국가, 엄마 출신국가, 아빠 출신국가
columns_to_update = ['Birth_Country', 'Birth_Country (Mother)', 'Birth_Country (Father)']

# 각 열에 대해 국가를 대륙으로 매핑
for column in columns_to_update:
    train_data[column] = train_data[column].map(country_continent_dict)

# Remove rows where 'Age' is below 17 or above 75
train_data = train_data[train_data['Age'].between(17, 75)]

# Remove rows with 'Employment Status' as 'Not Working' or 'Seeking Full-Time'
train_data = train_data[~train_data['Employment_Status'].isin(['Not Working', 'Seeking Full-Time'])]

# Drop the 'Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status' columns
train_data.drop(['Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status'], axis=1, inplace=True)

# Map 'Gender' values from 'M' and 'F' to 0 and 1, respectively
train_data['Gender'] = train_data['Gender'].map({'M': 0, 'F': 1})

# Consolidate education levels and rename as specified
education_map = {
    'High graduate': 'High', 'High Senior': 'High', 
    'High Junior': 'High', 'High Sophomore': 'High',
    'Elementary (5-6)': 'Elementary(1-6)', 'Elementary (1-4)': 'Elementary(1-6)',
    'Kindergarten': 'Baby', 'Children': 'Baby'
}
train_data['Education_Status'] = train_data['Education_Status'].replace(education_map)
columns=[
    'Education_Status',
    'Employment_Status',
    'Industry_Status',
    'Occupation_Status',
    'Race',
    'Hispanic_Origin',
    'Martial_Status',
    'Household_Summary',
    'Citizenship',
    'Birth_Country',
    'Birth_Country (Father)',
    'Birth_Country (Mother)',
    'Tax_Status'
]
edit_train_data=pd.get_dummies(train_data,columns=columns,dtype=int)
edit_train_data = edit_train_data.drop(columns=['ID'])
edit_train_data = edit_train_data.drop(columns=['Income'])
edit_train_data.to_csv("editedTrain.csv", index=False)

---


# 작업 영역 시작

#### RMSE를 줄이기 위해 이후 작업이 필요한 부분

#### 알맞은 모델과 파라미터를 조정해 가며 비교 필요

#### 모델 학습


In [8]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


ready_X = edit_train_data
ready_y = train_data['Income']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(ready_X, ready_y, test_size=0.2, random_state=42)

# 모델 초기화 및 학습
rf = RandomForestRegressor(n_estimators=100, random_state=50)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=50)

#### 모델 성능 평가 (Training Data 이용)

In [9]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV

# 모델 학습
rf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = rf.predict(X_test)

# 성능 평가
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R²: {r2}")


RMSE: 720.8605932437393
R²: 0.05157198443552957


In [10]:

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV


In [11]:
# 데이터 분할은 이미 완료된 상태입니다(X_train, X_test, y_train, y_test).

# XGBoost 모델 초기화
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed=42)

# 모델 학습
xg_reg.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = xg_reg.predict(X_test)

# 성능 평가
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

print(f"Initial RMSE: {rmse}")


Initial RMSE: 723.891024019801


In [13]:
# 확장된 그리드 탐색을 위한 하이퍼파라미터 설정
param_grid_expanded = {
    'n_estimators': [100, 300, 500, 800],  # 트리의 수
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # 학습률
    'max_depth': [3, 4, 5, 6, 8],  # 트리의 최대 깊이
    'colsample_bytree': [0.3, 0.5, 0.7, 1.0],  # 각 트리의 컬럼 샘플링 비율
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],  # 각 트리에서 샘플링하는 데이터의 비율
    'min_child_weight': [1, 2, 5, 10],  # 결정 트리의 min_child_weight 파라미터
    'gamma': [0, 0.1, 0.2, 0.5],  # 트리의 리프 노드에서 추가로 분할을 결정하기 위한 최소 손실 감소
}

# GridSearchCV를 사용한 확장된 그리드 탐색
grid_search_expanded = GridSearchCV(estimator=xg_reg, param_grid=param_grid_expanded, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# 확장된 그리드 탐색 수행
grid_search_expanded.fit(X_train, y_train)

# 최적의 모델 찾기
best_grid_expanded = grid_search_expanded.best_estimator_

# 테스트 데이터에 대한 예측 수행
y_pred_optimized_expanded = best_grid_expanded.predict(X_test)

# 성능 평가
mse_optimized_expanded = mean_squared_error(y_test, y_pred_optimized_expanded)
rmse_optimized_expanded = mse_optimized_expanded ** 0.5
print(f"Expanded Optimized RMSE: {rmse_optimized_expanded}")


Fitting 3 folds for each of 25600 candidates, totalling 76800 fits
[CV] END colsample_bytree=0.3, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.3, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.3, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.6; total time=   0.2s
[CV] END colsample_bytree=0.3, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.7; total time=   0.2s
[CV] END colsample_bytree=0.3, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.7; total time=   0.2s
[CV] END colsample_bytree=0.3, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.7; total time=   0.2s
[CV] END colsample_bytree=0.3, gamma=0, learning_rate=0.01, max

KeyboardInterrupt: 

# 작업 영역 끝

---


#### 제출 파일 만들기

Test Data 불러오기


In [59]:
testData = pd.read_csv("EditedTest.csv")

전처리가 원료된 testData를 원핫 인코딩

In [60]:

row_to_zero = testData[(testData["Age"] < 17) | 
                      (testData["Age"] > 75) | 
                      (testData["Employment_Status"] == "Not Working") | 
                      (testData["Employment_Status"] == "Seeking Full-Time")].index

# 'df_encoded'는 훈련 데이터에 대해 이미 원핫 인코딩이 수행된 DataFrame입니다.
# 'train_cols'는 원핫 인코딩된 훈련 데이터의 열 순서입니다.
train_cols = edit_train_data.columns

# 'columns' 리스트에 지정된 열에 대해 testData 데이터프레임을 원핫 인코딩합니다.
edited_testData = pd.get_dummies(testData, columns=columns, dtype=int)

# 훈련 데이터에만 있는 열을 찾습니다.
missing_cols = set(train_cols) - set(edited_testData.columns)

# 훈련 데이터에만 있는 열을 테스트 데이터에 추가하고, 해당 열의 값을 0으로 설정합니다.
for col in missing_cols:
    edited_testData[col] = 0

# 테스트 데이터에서 훈련 데이터에 없는 열을 찾습니다.
extra_cols = set(edited_testData.columns) - set(train_cols)

# 테스트 데이터에서 훈련 데이터에 없는 열을 제거합니다.
edited_testData = edited_testData.drop(columns=extra_cols)

# 테스트 데이터의 열 순서를 훈련 데이터의 열 순서와 동일하게 재정렬합니다.
edited_testData = edited_testData[train_cols]

# 이제 'edited_testData'는 원핫 인코딩된 테스트 데이터이며, 열 순서가 'df_encoded'와 일치합니다.


# 여기서 알맞은 모델을 사용해서 y_pred를 예측해야한다 (작업영역임)

예측값 생성해서 y_pred에 저장

In [61]:
y_pred = grid_search.predict(edited_testData) 
for i in range(len(y_pred)):
    if i in row_to_zero:
        y_pred[i]=0

test.csv 파일에 있는 데이터를 예측하여 submissionFile.csv 파일로 저장


In [64]:
# ID 생성
ids = [f'TEST_{i:04d}' for i in range(len(y_pred))]

# 데이터프레임 생성
df = pd.DataFrame({
    'ID': ids,
    'Income': y_pred
})

# CSV 파일로 저장
df.to_csv('submissionFile.csv', index=False)